In [41]:
import numpy as np 
import matplotlib.pyplot as plt
import numba as nb
import cython as cy
import scipy
import const 
from scipy.special import gamma
from scipy.special import kv
from scipy import *
from scipy.integrate import odeint
import math as m
import mpmath

In [42]:
def min(a, b):
    if a < b:
        return a
    else:
        return b


In [43]:
#Définition des fonctions de Bessel de première espèce (avec une limite finie en 0)

K1p = lambda x: mpmath.besselk(1,x)
K2p = lambda x: mpmath.besselk(2,x)
K3p = lambda x: mpmath.besselk(3,x)

K1 = np.vectorize(K1p)
K2 = np.vectorize(K2p)
K3 = np.vectorize(K3p)

'''
def K1(x) :
    return kv(1, x)

def K2(x) :
    return kv(2, x)

def K3(x) :
    return kv(3, x)

'''

def gamma_maj(ze):
    return float((K3(ze)/K2(ze) - 1/ze).real)

def gamma_chapeau_prime(ze):
    a = ze*(3*K3(ze) + K1(ze) - 4*K2(ze))
    return float((1 + 4*K2(ze)/a).real)

gamma_maj =np.vectorize(gamma_maj)
gamma_chapeau_prime = np.vectorize(gamma_chapeau_prime)





In [44]:
#Def des constantes 

tetav=20*3.14/180
c=3*10**8
mp = 1.67e-27
sh_max=100
E=10**43
n0=10**(3)  #let's say 1 particle by cubic cm



In [45]:
def mu(teta, tetav, phi):
    return np.sin(teta) * np.sin(tetav) * np.cos(phi) + np.cos(teta) * np.cos(tetav)


def C_BM(E, n0):
    return np.sqrt(17 * E / (8 * np.pi * n0 * const.m_p * const.celer**5))

def C_ST(E, n0):
    return (2/5) * 1.15 * (E / (n0 * const.m_p * const.celer**5))**(1/5)




#TODO: Petit ou grand t ici ? 

def radius(T, teta, tetav, phi, E, n0, sh_max):
    C_BM_val = C_BM(E, n0)
    C_ST_val = C_ST(E, n0)
    mu_val = mu(teta, tetav, phi)
    def f(R, t):
        t =T+ mu_val * R / const.celer
        mine = np.minimum((C_BM_val**2) * (t) ** (-3) + (C_ST_val**2) * (t) ** (-6 / 5), sh_max**2)
        return np.sqrt(mine / (1 + mine)) * const.celer
    
    Rp = -T*const.celer/mu_val
    Rs = 1e25
    cpt = 0
    while abs(Rp - Rs)/abs(Rs) > 10 ** (-2) and cpt < 20:
        cpt += 1
        #print(Rs)
        time1 = [0,T+mu_val*Rs/const.celer]
        sol = odeint(f,y0=[-T*const.celer/mu_val], t=time1)
        Rp = Rs
        Rs = sol[1][0]
    return Rs

def shock_speed(E, n0, sh_max, T, teta, tetav, phi):
    mu_val =mu(teta, tetav, phi)
    radius_value = radius(T, teta, tetav, phi, E, n0, sh_max)
    t = T + mu_val * radius_value / const.celer
    mine = min((C_BM(E, n0)**2) * (t) ** (-3) + (C_ST(E, n0)**2) * (t) ** (-6/5), sh_max**2)
    beta_sh = np.sqrt(mine / (1 + mine))
    return beta_sh












In [46]:
# Détermination de Ksi, car il est défini implicitement ici. 
#Find the rpoblem with ksi ici
from scipy.optimize import fsolve

def eq(gamma_sh,ze):
    a = (gamma_maj(ze) + 1)*(1 + gamma_chapeau_prime(ze)*(gamma_maj(ze) - 1))**2
    b = 2 + gamma_chapeau_prime(ze)*(2 - gamma_chapeau_prime(ze)*(gamma_maj(ze) - 1))
    return gamma_sh**2 - a/b

def sol(gamma_sh): #Sol is zeta
    def func(ze):
        return eq(gamma_sh, ze)
    solve = fsolve(func, 1)
    return solve[0]

sol_vect = np.vectorize(sol)
g_m = np.vectorize(gamma_maj)
g_c = np.vectorize(gamma_chapeau_prime)


#We can now calculate values of beta's

def beta_u(E, n0, sh_max, T, teta, tetav, phi):
    return shock_speed(E, n0, sh_max, T, teta, tetav, phi)

def beta(E, n0, sh_max,T, teta, tetav, phi):
    beta_sh = shock_speed(E, n0, sh_max,T, teta, tetav, phi)
    zeta = sol(beta_sh)
    return np.sqrt(1 - 1/(gamma_maj(zeta) ** 2))

def beta_d(E, n0, sh_max,T, teta, tetav, phi):
    beta_sh = shock_speed(E, n0, sh_max, T, teta, tetav, phi)
    return (beta_sh - beta(E, n0, sh_max,T, teta, tetav, phi)) / (1 - beta(E, n0, sh_max,T, teta, tetav, phi) * beta_sh)

def p(E, n0, sh_max, T, teta, tetav, phi):
    beta_u_val = beta_u(E, n0, sh_max, T, teta, tetav, phi)
    beta_d_val = beta_d(E, n0, sh_max, T, teta, tetav, phi)
    a = 3*beta_u_val - 2*beta_u_val*(beta_d_val**2) + beta_d_val**3
    b = beta_u_val - beta_d_val
    return a/b - 2

def gamma_shf(E, n0, sh_max,T, teta, tetav, phi):
    beta_sh = shock_speed(E, n0, sh_max, T, teta, tetav, phi)
    return np.sqrt(1/(1 - beta_sh**2))

def n_prime(gamma_sh, n0):
    return 4*gamma_maj(sol(gamma_sh))*n0

def e_i_p(n0, gamma_sh):
    return const.m_p*((const.celer)**2) * (gamma_maj(sol(gamma_sh)) - 1)*n_prime(n0,gamma_sh)

epsi_b = 4.12*(1/np.power(10,5))
def gamma_prime_c(n0, epsilon_B, gamma_sh,t):
    return (3 * const.m_e * const.celer* gamma_maj(sol(gamma_sh))) / (4 * const.sigma_T * epsilon_B *e_i_p(gamma_sh) * t*n0)


def gamma_prime_m(E, n0, sh_max, T, gamma_sh, teta, tetav, phi):
    p_val = p(E, n0, sh_max, T,teta, tetav, phi)
    arg = ((p_val - 2)*const.m_p/(p_val - 1)*const.m_e)*const.epsi_e*(gamma_maj(sol(gamma_sh)) - 1)
    return max(1, arg)

def B_prime(epsilon_B, gamma_sh, n0):
    return (np.sqrt(8*const.pi*epsilon_B*e_i_p(n0, gamma_sh)))

def nu_prime_m(epsilon_B, gamma_sh, E, n0, sh_max, T, teta, tetav, phi):
    return 3/16 * gamma_prime_m(E, n0, sh_max, T, gamma_sh, teta, tetav, phi)**2 * const.q_e*B_prime(epsilon_B, gamma_sh,n0) / (const.m_e * const.celer)

def nu_prime_c(n0,epsilon_B, t, gamma_sh):
    return 3/16 * gamma_prime_c(n0,epsilon_B,gamma_sh, t)**2 * const.q_e *B_prime(epsilon_B, gamma_sh, n0) / (const.m_e * const.celer)


In [47]:
def power_I(x,y):
    return 1/np.power(x,y)

#Rapport de comparaison pour les refroidissemts entre nu_prim_c et nu_prim_m
def Kappa(n0,epsilon_B,t,gamma_sh,E,T,teta, tetav, phi):
    return nu_prime_c(n0, epsilon_B, gamma_sh,t)/nu_prime_m(epsilon_B, gamma_sh, E, n0, sh_max, T, teta, tetav, phi)

def n_prime_r(gamma_sh, E, n0, sh_max, T):
    p_val = p(E, n0, sh_max, T)
    return n_prime(gamma_sh,n0)*min(1,((p_val - 2)*const.m_p/(p_val - 1)*const.m_e)*const.epsi_e*(gamma_maj(sol(gamma_sh)) - 1))

def FastcEmission(n0,x,p,epsilon_B,t,gamma_sh,E,T, teta, tetav, phi):
    if x > Kappa(n0,epsilon_B,t,gamma_sh,E):
        return np.power(Kappa(n0,epsilon_B,t,gamma_sh,E,T,teta, tetav, phi),-1/3)*np.power(x,1/3)
    elif x >= Kappa(n0,epsilon_B,t,gamma_sh,E,T,teta, tetav, phi) and x <1:
        return np.power(Kappa(n0,epsilon_B,t,gamma_sh,E,T,teta, tetav, phi),1/2)*power_I(x,-1/2)
    else:
         return power_I(Kappa(n0,epsilon_B,t,gamma_sh,E,T,teta, tetav, phi),-1/2)*power_I(x,-p/2)

#Cas du slow-cooling
def SlowcEmission(x,p,epsilon_B,t,gamma_sh,n0,E, T, teta, tetav, phi):
    if x < 1:
        return np.power(x,1/3)
    elif x>=1 and x < Kappa(n0,epsilon_B,t,gamma_sh,E,T,teta, tetav, phi):
        return power_I(x,-(p-1)/2)
    else:
        return power_I(Kappa(n0,epsilon_B,t,gamma_sh,E,T,teta, tetav, phi),-1/2)*power_I(x,-p/2)




#Fonction d'émission
def Emission_function(n0,x,epsilon_B,t,gamma_sh,p,E,T,teta, tetav, phi):
    if Kappa(n0,epsilon_B,t,gamma_sh,E,T,teta, tetav, phi) > 1:
        return SlowcEmission(x,p,epsilon_B,t,gamma_sh,n0,E)
    else:
        return FastcEmission(n0,x,p,epsilon_B,t,gamma_sh,E)


def fact_denorm(beta_u,beta_d,n0,gamma_sh,epsilon_b):
    return 0.88*(256*(p(beta_u,beta_d) - 1)*(const.q_e**3)*n_prime_r(n0,gamma_sh,beta_u,beta_d)
                 *B_prime(epsilon_b,gamma_sh,n0)/27*(3*(p(beta_u,beta_d) - 1)*const.m_e*(const.celer**2)))

In [48]:

def p_tildt(gamma_sh, n0, epsilon_B, t, E, sh_max,T, teta, tetav, phi):
    p_val = p(E, n0, sh_max, T, teta, tetav, phi)
    if gamma_prime_m(E, n0, sh_max, T, gamma_sh, teta, tetav, phi) >= gamma_prime_c(n0, epsilon_B, gamma_sh, t):
        return p_val 
    else:
        beta_u_val = beta_u(E, n0, sh_max, T, teta, tetav, phi)
        beta_d_val = beta_d(E, n0, sh_max, T, teta, tetav, phi)
        return (3 * beta_u_val - 2 * beta_u_val * beta_d_val ** 2 + beta_d_val ** 3) / (beta_u_val - beta_d_val) - 2

def c_11(nu_prime, gamma_sh, n0, epsilon_B, t, E, sh_max,T, teta, tetav, phi):
    nu_prime_c_val = nu_prime_c(n0, epsilon_B, t, gamma_sh)
    nu_prime_m_val = nu_prime_m(epsilon_B, gamma_sh, E, n0, sh_max, T, teta, tetav, phi)
    gamma_prime_m_val = gamma_prime_m(E, n0, sh_max, T, gamma_sh, teta, tetav, phi)
    gamma_prime_c_val = gamma_prime_c(n0, epsilon_B, gamma_sh, t)
    n_prime_r_val = n_prime_r(gamma_sh, E, n0, sh_max, T)
    b_prime = B_prime(epsilon_B, gamma_sh,n0)
    p_tildt_val = p_tildt(gamma_sh, n0, epsilon_B, t, E, sh_max,T, teta, tetav, phi)
    
    nu_min = min(nu_prime_m_val, nu_prime_c_val)
    gamma_min = min(gamma_prime_m_val, gamma_prime_c_val)
    return 2 ** 6 *const.pi ** (5/6) * (p_tildt_val + 2) * (p_tildt_val - 1) * const.q_e * n_prime_r_val / (15 * (3*p_tildt_val + 2) * gamma(5 / 6) * gamma_min ** 5 * b_prime) * (nu_prime / nu_min) ** (-5/3)

def c_14(nu_prime, gamma_sh, n0, epsilon_B, t, E, sh_max,T, teta, tetav, phi):
    nu_prime_c_val = nu_prime_c(n0, epsilon_B, t, gamma_sh)
    nu_prime_m_val = nu_prime_m(epsilon_B, gamma_sh, E, n0, sh_max, T, teta ,tetav, phi)
    gamma_prime_m_val = gamma_prime_m(E, n0, sh_max, T, gamma_sh, teta, tetav, phi)
    gamma_prime_c_val = gamma_prime_c(n0, epsilon_B, gamma_sh, t)
    n_prime_r_val = n_prime_r(gamma_sh, E, n0, sh_max, T)
    b_prime = B_prime(epsilon_B, gamma_sh, n0)
    p_tildt_val = p_tildt(gamma_sh, n0, epsilon_B, t, E, sh_max, T, teta, tetav, phi)
    nu_min = min(nu_prime_m_val, nu_prime_c_val)
    gamma_min = min(gamma_prime_m_val, gamma_prime_c_val)
    return (2 ** ((3*p_tildt_val + 8) / 2) * (p_tildt_val - 1) * gamma(3/2 + p_tildt_val/4) * gamma(1/6 + p_tildt_val/4) * const.q_e * n_prime_r_val ) / (3**(3/2) * const.pi **((p_tildt_val + 1 )/2) * gamma(2 + p_tildt_val/4) * gamma_min ** 5 *b_prime) *(nu_prime / nu_min) ** (-(p_tildt_val + 4) / 2)

def nu_prime_zero(gamma_sh, n0, epsilon_B, t, E, sh_max,T, teta, tetav, phi):
    nu_prime_c_val = nu_prime_c(n0, epsilon_B, t, gamma_sh)
    nu_prime_m_val = nu_prime_m(epsilon_B, gamma_sh, E, n0, sh_max, T, teta, tetav, phi)
    nu_prime_min = min(nu_prime_m_val, nu_prime_c_val)
    p_tildt_val = p_tildt(gamma_sh, n0, epsilon_B, t, E, sh_max,T, teta, tetav, phi)
    return ((5*(3 * p_tildt_val +2) * gamma(3/2 + p_tildt_val/4) *gamma(11/6 + p_tildt_val/4) * gamma(1/6 + p_tildt_val/4) * gamma(5/6)) / (p_tildt_val + 2) * gamma(2 + p_tildt_val/4))** (6 / (3*p_tildt_val+2)) * ((2**( 3 * (3 * p_tildt_val -4))) / (27 * const.pi ** (3*p_tildt_val + 8) )) ** (1 / (3*p_tildt_val + 2)) * nu_prime_min

def alpha_prime_nu_prime(nu_prime, gamma_sh, n0, epsilon_B, t, E, sh_max, T, teta, tetav, phi):
    p_tildt_val = p_tildt(gamma_sh, n0, epsilon_B, t, E, sh_max,T, teta, tetav, phi)
    nu_zero = nu_prime_zero(gamma_sh, n0, epsilon_B, t, E, sh_max,T)
    if nu_prime < nu_zero:
        alpha_prime_nu_prime_zero = c_11(nu_prime, gamma_sh, n0, epsilon_B, t, E, sh_max,T)
        eta =  -5/3
    else : 
        alpha_prime_nu_prime_zero = c_14(nu_prime, gamma_sh, n0, epsilon_B, t, E, sh_max,T)
        eta = -(p_tildt_val + 4) / 2
    return alpha_prime_nu_prime_zero * (nu_prime / nu_zero) ** eta

def nu_prime(nu, xi, beta, mu):
    return gamma_maj(xi) * (1 - beta * mu) * nu 




In [49]:


def alpha_nu(E, n0, sh_max,T, nu_prime, epsilon_B, t, gamma_sh, teta, tetav, phi):
    mu_val = mu(teta, tetav, phi)
    beta_sh = shock_speed(E, n0, sh_max, T, teta, tetav, phi)
    beta_val = beta(E, n0, sh_max,T, teta, tetav, phi)
    alpha_prime_nu_prime_val = alpha_prime_nu_prime(nu_prime, gamma_sh, n0, epsilon_B, t, E, sh_max,T, teta, tetav, phi)
    zeta = sol(beta_sh)
    alpha_nu = gamma_maj(zeta)*(1-beta_val*mu_val)*alpha_prime_nu_prime_val
    return alpha_nu

def delta_s(R, mu, E, n0, sh_max,T, teta, tetav, phi):
    beta_sh = shock_speed(E, n0, sh_max, T, teta, tetav, phi)
    zeta = sol(beta_sh)
    return R/((12*np.abs(mu - beta_sh)*gamma_maj(zeta))**2)

def tan_nu(R, E, n0, sh_max,T, nu_prime, epsilon_B, t, gamma_sh, teta, tetav, phi):
    mu_val = mu(teta, tetav, phi)
    alpha_nu_val = alpha_nu(E, n0, sh_max,T, nu_prime, epsilon_B, t, gamma_sh, teta, tetav, phi)
    delta_s_val = delta_s(R, mu_val, E, n0, sh_max,T, teta, tetav, phi)
    return alpha_nu_val * delta_s_val

In [50]:
#Calcul de l'Intégrande pour un T donné:
def integrand(nu,n0, epsi_b,tetav, T, teta,phi, E, sh_max, ):
    radius_val = radius(T, teta, tetav, phi, E, n0, sh_max)
    t = T + mu(teta, tetav, phi)*radius_val/const.celer # Pour un certain angle polaire, pour un certain temps de l'observateur, on détermine t
    gamma_sh_val = gamma_shf(E,n0,sh_max,T, teta, tetav, phi)
    zeta_val = sol(gamma_sh_val)
    p_val = p(E, n0, sh_max, T, teta, tetav, phi)
    mu_val = mu(teta, tetav, phi)
    nu_p_val = nu*gamma_maj(zeta_val)*(1-beta(E,n0,sh_max,T, teta, tetav, phi)*mu(teta,tetav,phi)) #valeur de nu' qui correspond à la valeur de nu en argument
    x_val = nu_p_val/nu_prime_m(epsi_b, gamma_sh_val, E, n0, sh_max, T, teta, tetav, phi)
    energ_normalized = Emission_function(n0,x_val,epsi_b,t,gamma_sh_val,p_val,E,T, teta, tetav, phi)
    energ = energ_normalized*fact_denorm(beta_u(E, n0,sh_max,T), beta_d(E, n0,sh_max,T),n0,gamma_sh_val,epsi_b) #Calcul de l'énergie pour la fréquence nu'
    alpha_prime_nu_prime_val = alpha_prime_nu_prime(nu_p_val, gamma_sh_val, n0, epsi_b, t, E, sh_max,T,teta,tetav, phi)
    alpha_nu_val = alpha_nu(E, n0, sh_max,T, nu_p_val, epsi_b, t, gamma_sh_val, teta, tetav, phi)
    delta_s_val = delta_s(radius_val, mu_val, E, n0, sh_max,T, teta, tetav, phi)
    tau_nu = alpha_nu_val * delta_s_val #Pour la suite on calcul la partie liée aux agnles et à la vitesse du choc
    angular_sh = np.abs(mu(teta, tetav, phi) - shock_speed(E,n0,sh_max,T,teta, tetav, phi))*np.power(radius(T,teta,tetav,phi,E,n0,sh_max),2)/(1-mu(teta, tetav, phi)*shock_speed(E,n0,sh_max,T,teta, tetav, phi))*np.power(gamma_maj(zeta_val)*(1 - beta(E,n0,sh_max,T)*mu(teta, tetav,phi)),3)
    return angular_sh*energ*(1- m.exp( -tau_nu))/alpha_prime_nu_prime_val


nu = 3*10**10
n0=10**(3)
epsi_b = 4.12*(1/np.power(10,5)) 
tetav = 20*3.14/180
T = 10**5
teta = 20*3.14/180
phi = 20*3.14/180
E = 10**25
sh_max = 100
print(integrand(nu,n0, epsi_b,tetav, T, teta,phi, E, sh_max))


    
    

c:\miniconda\envs\ue22\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in power


TypeError: e_i_p() missing 1 required positional argument: 'gamma_sh'